In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score
from transformers import CLIPProcessor, CLIPModel, CLIPVisionModel
from tqdm import tqdm
from PIL import ImageFile

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda')

In [3]:
# Путь к данным animals
train_path = 'D:\\ProgPrj\\dsProjects\\gazprom-media\\ml\\train'
test_path = 'D:\\ProgPrj\\dsProjects\\gazprom-media\\ml\\test'

In [4]:
# Преобразования для тренировочного и валидационного наборов данных
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomApply(torch.nn.ModuleList([transforms.ColorJitter()]), p=0.25),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomRotation(degrees=(-10, 10)),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.1, value='random')
])

In [5]:
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [6]:
# Определение модели
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
model.to(DEVICE)

C:\Users\Geo\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\Geo\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

In [7]:
# Оптимизатор и функция потерь
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Scheduler для оптимизации
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)

In [8]:
# Загрузка данных
train_dataset = datasets.ImageFolder(train_path, transform=train_transform)
val_dataset = datasets.ImageFolder(test_path, transform=val_transform)

In [9]:
print(len(train_dataset.class_to_idx))

101


In [10]:
batch_size = 96

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

In [11]:
# Скалер для смешанной точности
scaler = torch.cuda.amp.GradScaler()

# Обучение модели
num_epochs = 100

for epoch in range(num_epochs):
    ImageFile.LOAD_TRUNCATED_IMAGES = True
    model.train()
    running_loss = 0.0
    train_loader_tqdm = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}")

    for images, labels in train_loader_tqdm:
        images, labels = images.to(DEVICE, non_blocking=True), labels.to(DEVICE, non_blocking=True)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model.get_image_features(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * images.size(0)
        train_loader_tqdm.set_postfix(loss=loss.item())

    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch {epoch+1}/{num_epochs} Train Loss: {epoch_loss:.4f}")

Training Epoch 1/100: 100%|██████████| 790/790 [01:40<00:00,  7.85it/s, loss=4.07]


Epoch 1/100 Train Loss: 4.2582


Training Epoch 2/100: 100%|██████████| 790/790 [01:30<00:00,  8.72it/s, loss=4.45]


Epoch 2/100 Train Loss: 3.6059


Training Epoch 3/100: 100%|██████████| 790/790 [01:33<00:00,  8.42it/s, loss=3.29]


...

Epoch 98/100 Train Loss: 0.1428


Training Epoch 99/100: 100%|██████████| 790/790 [01:33<00:00,  8.46it/s, loss=0.0061]


Epoch 99/100 Train Loss: 0.1373


Training Epoch 100/100: 100%|██████████| 790/790 [01:32<00:00,  8.57it/s, loss=0.000996]

Epoch 100/100 Train Loss: 0.1363


In [12]:
# Validation loop
model.eval()
val_loss = 0
val_preds = []
val_labels = []
with torch.no_grad():
    for images, labels in tqdm(val_loader, desc="Validation"):
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model.get_image_features(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()

        preds = outputs.argmax(dim=1)
        val_preds.extend(preds.cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

val_loss /= len(val_loader)
val_accuracy = accuracy_score(val_labels, val_preds)
val_f1 = f1_score(val_labels, val_preds, average='weighted')

print("Validation Results")
print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1: {val_f1:.4f}")

# Step scheduler based on validation loss
scheduler.step(val_loss)

Validation: 100%|██████████| 264/264 [00:32<00:00,  8.14it/s]

Validation Results
Val Loss: 2.9786, Val Accuracy: 0.5405, Val F1: 0.5359
